https://stashinvest.atlassian.net/browse/DSA-712


```sql
select
  sub.user_id as stash_user_id,
  date_trunc('month',act.execution_date)
from
  source_subscriptions.subscription_actions as act left join
  source_subscriptions.subscriptions as sub on sub.id = act.subscription_id
where act.action='cancel'


with user_cancel_count as (
select
  sub.user_id as stash_user_id, count(*) as cancel_count
from
  source_subscriptions.subscription_actions as act left join
  source_subscriptions.subscriptions as sub on sub.id = act.subscription_id
where act.action='cancel' and act.execution_date >= '2020-01-01'
group by 1
)
select cancel_count, count(*) from user_cancel_count group by 1


```


---


```sql
select action from source_subscriptions.subscription_actions group by 1

activate
cancel
create
downgrade
end_free_trial
reactivate
update
upgrade

select to_aasm_state from source_pg_main.account_state_transitions group by 1

back_office
closed
complete
errored
impeded
in_activation_process
in_closing_process
in_reopen_process
inactive
incomplete
indeterminate
pending
queued_for_apex
rejected
unsupported

```


---

```sql


with all_ob2 as (
select
  sub.user_id as stash_user_id,
  max(act.execution_date) as churned_date
from
  source_subscriptions.subscription_actions as act left join
  source_subscriptions.subscriptions as sub on sub.id = act.subscription_id
where act.action='cancel'
group by 1
),

all_ob1_some_ob2 as (
select
  account.user_id as stash_user_id,
  date(max(transition.created_at)) as churned_date
from
  source_pg_main.accounts as account,
  source_pg_main.account_state_transitions as transition
where
  account.id = transition.account_id and
  to_aasm_state in ('closed','in_closing_process')
group by 1
)

select
  count(sub.stash_user_id) as users,
  sum(balance.invest_balance) as balance
from
  mart.subscriptions as sub left join
  all_ob2 using (stash_user_id) left join
  all_ob1_some_ob2 using (stash_user_id) left join
  integrated_users.platform_start_of_days as balance
on
  stash_user_id = balance.user_id and
  coalesce(all_ob2.churned_date,all_ob1_some_ob2.churned_date)  = balance.file_date
where
  sub.last_revenue_at between '2020-09-01' and '2021-08-31'





with last_transition_at as (
select
  account.user_id as stash_user_id,
  max(transition.created_at) as last_transition_at
from
  source_pg_main.accounts as account,
  source_pg_main.account_state_transitions as transition
where
  account.id = transition.account_id
  -- and transition.to_aasm_state = 'closed'
group by 1
)

select
  count(stash_user_id) as users,
  sum(balance.invest_balance) as balance
from
  mart.subscriptions as sub left join
  last_transition_at using (stash_user_id) left join
  integrated_users.platform_start_of_days as balance
on
  stash_user_id = balance.user_id and
  date(last_transition_at) = balance.file_date
where
  sub.last_revenue_at between '2020-09-01' and '2021-08-31'


```

---

```sql

select
  count(sub.stash_user_id) as users,
  sum(balance.invest_balance) as balance
from
  mart.subscriptions as sub left join
  integrated_users.platform_start_of_days as balance
on
  sub.stash_user_id = balance.user_id and
  sub.last_revenue_at = balance.file_date
where
  sub.last_revenue_at between '2020-09-01' and '2021-08-31'


with

sub_action as ( select
                  subscription_id,
                  action
                  created_at as initiated_at,
                  execution_date as completed_at
                from source_subscriptions.subscription_actions),

sub as (        select
                  user_id as stash_user_id,
                  id as subscription_id
                from source_subscriptions.subscriptions),

acc_action as ( select
                  account_id,
                  to_aasm_state as action,
                  created_at as completed_at
                from source_pg_main.account_state_transitions),

acc as (        select
                  user_id as stash_user_id,
                  id as account_id
                from source_pg_main.accounts),

```